In [2]:
# example of training a stable gan for generating a handwritten digit
from os import makedirs
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.initializers import RandomNormal
from matplotlib import pyplot

# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# downsample to 14x14
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 7x7
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model
def define_generator(latent_dim):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load mnist images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# select all of the examples for a given class
	selected_ix = trainy == 8
	X = X[selected_ix]
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n_samples, 1))
	return X, y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	pyplot.savefig('results_baseline/generated_plot_%03d.png' % (step+1))
	pyplot.close()
	# save the generator model
	g_model.save('results_baseline/model_%03d.h5' % (step+1))

# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
	# plot loss
	pyplot.subplot(2, 1, 1)
	pyplot.plot(d1_hist, label='d-real')
	pyplot.plot(d2_hist, label='d-fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	# plot discriminator accuracy
	pyplot.subplot(2, 1, 2)
	pyplot.plot(a1_hist, label='acc-real')
	pyplot.plot(a2_hist, label='acc-fake')
	pyplot.legend()
	# save plot to file
	pyplot.savefig('results_baseline/plot_line_plot_loss.png')
	pyplot.close()

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
	# calculate the number of batches per epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the total iterations based on batch and epoch
	n_steps = bat_per_epo * n_epochs
	# calculate the number of samples in half a batch
	half_batch = int(n_batch / 2)
	# prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
		# generate 'fake' examples
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
			(i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
		# record history
		d1_hist.append(d_loss1)
		d2_hist.append(d_loss2)
		g_hist.append(g_loss)
		a1_hist.append(d_acc1)
		a2_hist.append(d_acc2)
		# evaluate the model performance every 'epoch'
		if (i+1) % bat_per_epo == 0:
			summarize_performance(i, g_model, latent_dim)
	plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)

# make folder for results
makedirs('results_baseline', exist_ok=True)
# size of the latent space
latent_dim = 50
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

11493376/11490434 [==============================] - 3s 0us/step
(5851, 28, 28, 1)
>1, d1=0.696, d2=0.694 g=0.692, a1=37, a2=0
>2, d1=0.658, d2=0.696 g=0.690, a1=100, a2=0
>3, d1=0.626, d2=0.699 g=0.687, a1=100, a2=0
>4, d1=0.594, d2=0.704 g=0.682, a1=100, a2=0
>5, d1=0.559, d2=0.713 g=0.674, a1=100, a2=0
>6, d1=0.522, d2=0.727 g=0.661, a1=100, a2=0
>7, d1=0.485, d2=0.749 g=0.643, a1=100, a2=0
>8, d1=0.458, d2=0.781 g=0.620, a1=100, a2=0
>9, d1=0.427, d2=0.820 g=0.596, a1=100, a2=0
>10, d1=0.406, d2=0.851 g=0.582, a1=100, a2=0
>11, d1=0.395, d2=0.859 g=0.584, a1=100, a2=0
>12, d1=0.391, d2=0.844 g=0.603, a1=100, a2=0
>13, d1=0.401, d2=0.804 g=0.640, a1=100, a2=0
>14, d1=0.410, d2=0.755 g=0.687, a1=100, a2=0
>15, d1=0.390, d2=0.698 g=0.754, a1=100, a2=17
>16, d1=0.411, d2=0.634 g=0.823, a1=100, a2=100
>17, d1=0.401, d2=0.576 g=0.903, a1=100, a2=100
>18, d1=0.401, d2=0.519 g=0.991, a1=100, a2=100
>19, d1=0.413, d2=0.464 g=1.085, a1=100, a2=100
>20, d1=0.394, d2=0.413 g=1.186, a1=100, a2=

>172, d1=0.695, d2=0.646 g=0.858, a1=56, a2=76
>173, d1=0.704, d2=0.635 g=0.839, a1=54, a2=75
>174, d1=0.667, d2=0.662 g=0.846, a1=60, a2=62
>175, d1=0.661, d2=0.667 g=0.848, a1=62, a2=64
>176, d1=0.648, d2=0.671 g=0.841, a1=68, a2=67
>177, d1=0.687, d2=0.642 g=0.842, a1=57, a2=79
>178, d1=0.684, d2=0.640 g=0.834, a1=59, a2=75
>179, d1=0.682, d2=0.670 g=0.835, a1=57, a2=56
>180, d1=0.679, d2=0.663 g=0.836, a1=53, a2=62
>181, d1=0.694, d2=0.670 g=0.827, a1=46, a2=67
>182, d1=0.683, d2=0.672 g=0.825, a1=56, a2=60
>183, d1=0.683, d2=0.679 g=0.816, a1=57, a2=64
>184, d1=0.685, d2=0.669 g=0.797, a1=67, a2=67
>185, d1=0.663, d2=0.666 g=0.819, a1=59, a2=67
>186, d1=0.698, d2=0.688 g=0.803, a1=51, a2=54
>187, d1=0.723, d2=0.684 g=0.815, a1=51, a2=59
>188, d1=0.702, d2=0.690 g=0.806, a1=56, a2=50
>189, d1=0.693, d2=0.668 g=0.798, a1=54, a2=67
>190, d1=0.685, d2=0.680 g=0.780, a1=64, a2=59
>191, d1=0.693, d2=0.700 g=0.804, a1=57, a2=54
>192, d1=0.692, d2=0.684 g=0.788, a1=56, a2=59
>193, d1=0.73

>347, d1=0.742, d2=0.715 g=0.723, a1=31, a2=34
>348, d1=0.730, d2=0.699 g=0.724, a1=20, a2=39
>349, d1=0.740, d2=0.708 g=0.724, a1=28, a2=42
>350, d1=0.752, d2=0.706 g=0.726, a1=26, a2=45
>351, d1=0.754, d2=0.695 g=0.726, a1=25, a2=53
>352, d1=0.735, d2=0.706 g=0.725, a1=29, a2=34
>353, d1=0.738, d2=0.709 g=0.733, a1=26, a2=39
>354, d1=0.741, d2=0.710 g=0.728, a1=20, a2=35
>355, d1=0.735, d2=0.693 g=0.731, a1=28, a2=48
>356, d1=0.727, d2=0.705 g=0.734, a1=32, a2=40
>357, d1=0.722, d2=0.699 g=0.733, a1=37, a2=50
>358, d1=0.737, d2=0.705 g=0.727, a1=29, a2=37
>359, d1=0.743, d2=0.693 g=0.718, a1=21, a2=56
>360, d1=0.741, d2=0.703 g=0.725, a1=25, a2=48
>361, d1=0.725, d2=0.699 g=0.728, a1=31, a2=39
>362, d1=0.733, d2=0.694 g=0.727, a1=28, a2=50
>363, d1=0.724, d2=0.702 g=0.724, a1=35, a2=35
>364, d1=0.740, d2=0.695 g=0.728, a1=25, a2=37
>365, d1=0.726, d2=0.708 g=0.724, a1=34, a2=32
>366, d1=0.728, d2=0.709 g=0.728, a1=29, a2=35
>367, d1=0.729, d2=0.700 g=0.727, a1=29, a2=51
>368, d1=0.71